In [ ]:
import geemap
import ee

Map = geemap.Map()
Map

In [ ]:
roi = geemap.shp_to_ee(r'H:/Monarch/Data/roi_prj_gcs.shp')
Map.addLayer(roi, {}, 'JX')
Map.center_object(roi)

In [ ]:
def modis_NDVI(img):
    img1 = img.normalizedDifference(['sur_refl_b02', 'sur_refl_b01'])
    return img1

In [ ]:
modis = ee.ImageCollection("MODIS/006/MOD09Q1").filterDate('2018', '2019')
landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI')\
            .filterDate('2018', '2019')

In [ ]:
Sentinel2 = ee.ImageCollection("COPERNICUS/S2").filterBounds(roi.geometry())\
            .sort('CLOUDY_PIXEL_PERCENTAGE', False).filterDate('2018-03-22', '2018-03-30')

In [ ]:
modis_names = modis.aggregate_array('system:id').getInfo()
landsat_names = landsat8.aggregate_array('system:id').getInfo()

In [ ]:
modis_name = modis_names[10]
landsat_name = landsat_names[10]
# modis_img = modis_NDVI(ee.Image(modis_name))
# landsat_img = ee.Image(landsat_name)
# NDVI_img = landsat_img.unmask(modis_img)

In [ ]:
Ndvi = Sentinel2.map(lambda img: img.normalizedDifference(['B8', 'B4']))

In [ ]:
Sentinel2.size().getInfo()

In [ ]:
modis_name

In [ ]:
ndviVis = {
  'min': 0,
  'max': 1,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}
Map.addLayer(Ndvi, ndviVis, 'n')

In [ ]:
f_name = 'NDVI_'+modis_name.split('/')[-1]
geemap.ee_export_image_to_drive(Ndvi, description=f_name,
                                folder='Image', scale=30)